In [34]:
import cv2
import matplotlib.pyplot as plt
import pytesseract
import re
import pandas as pd
from PIL import Image
import io
import os
import glob

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/sanghyunbyun/Desktop/rok/bin/ROKS-78e082836307.json"

plt.figure(figsize=(50, 40))
plt.style.use('dark_background')

images = glob.glob('./data/info/original/ex/*.PNG')
image_path = './data/info/original/ex/'
p3 = re.compile(image_path)

<Figure size 3600x2880 with 0 Axes>

In [2]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    # [START vision_python_migration_text_detection]
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in texts[0].bounding_poly.vertices])

    return texts[0].description

In [3]:
def change_to_number(s_power):
    list_of_numbers = re.findall(r'\d+', s_power)
    return int(''.join(list_of_numbers))

In [4]:
def change_to_list(chars):
    tmp_texts = list()
    s = ""
    for c in chars:
        if c != '\n':
            s += c
        else:
            tmp_texts.append(s)
            s = ""
    
    return tmp_texts

In [5]:
def convert_img(img, category, i):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
    gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

    ret, img_thresh = cv2.threshold(
        img_blurred, 
        160, #name : 180
        255,
        cv2.THRESH_BINARY
    )

    cv2.imwrite('./data/info/convert/' + category + '/' + str(i) + '.png',img_thresh)

In [85]:
# crop
# for image in images:
initial = Image.open(images[0])
width = initial.size[0]
height = initial.size[1]

image_name = re.sub(p3, '', initial.filename)
print(image_name)
        # 3200 x 1440 
        # name 0.295 0.15 0.635 0.2
        # kill 0.56 0.19 0.72 0.44
        # dead&support 0.20 0.23 0.80 0.98

        # 1334 x 750 
        # name 0.242 0.13 0.66 0.205
        # kill 0.57 0.19 0.78 0.44
        #name
    #     croped = initial.crop((width*0.242, height*0.13, width*0.66 , height*0.205))
    #     croped.save('./data/info/croped/name/' + image_name)

    #kill
#     croped = initial.crop((width*0.57, height*0.19, width*0.78, height*0.44))
#     croped.save('./data/info/croped/kill/' + image_name)

# #dead & support
croped = initial.crop((width*0.10, height*0.228, width*0.80, height*0.985))
croped.save('./data/info/croped/ds/' + image_name)

IMG_0549.PNG
